In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
data_format = 'token'
train_df = pd.read_csv(f{data_format}_train.tsv', sep='\t', header=None, names=['comment', 'label'])
train_df['is_valid'] = False
test_df = pd.read_csv(f'{data_format}_test.tsv', sep='\t', header=None, names=['comment', 'label'])
test_df['is_valid'] = True

df = pd.concat([train_df,test_df], sort=False).rename(columns={'comment': 'text'})
df.head()

,text,label,is_valid
0,ממש כואב ..... אני בוכה עם המשפחה שלא תדעו עוד...,0,False
1,איש יקר שלנו,0,False
2,כל הכבוד והמון בהצלחה,0,False
3,"תל חי , רובי . בכל העצב הזה היית קרן אור של ת...",0,False
4,נקי כפיים ובר לבב בהצלחה לך ולנו .,0,False


In [3]:
len(train_df), len(test_df), len(df)

(10244, 2560, 12804)

In [4]:
total = 0
train_comments = set(train_df.comment)
for comment in test_df.comment:
    if comment in train_comments: total += 1
print(total, total / len(test_df))

1297 0.506640625


In [5]:
dup_train_train = []
dup_train_test = []
dup_test_test = []
txt_set = set()
for i, row in tqdm(df.iterrows()):
    txt = row.text
    orig_dset = row.is_valid
    if txt in txt_set: continue
    txt_set.add(txt)
    curr_df = df[df.text == txt]
    dsets = curr_df.is_valid
    n_train, n_test = sum(dsets == False), sum(dsets == True)
    if orig_dset == False:
        if n_train > 1: dup_train_train.append(n_train-1)
        if n_test > 0: dup_train_test.append(n_test)
    elif orig_dset == True:
        if n_test > 1: dup_test_test.append(n_test-1)
sum(dup_train_train), sum(dup_train_test), sum(dup_test_test), sum(dup_train_train) + sum(dup_train_test) + sum(dup_test_test)

12804it [00:12, 1048.60it/s]


(2756, 1297, 132, 4185)

In [6]:
full = df
clean = df.drop_duplicates('text')
len(full), len(clean), len(full) - len(clean)

(12804, 8619, 4185)